In [ ]:
import os
import random
import warnings
import time
from pathlib import Path

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

import cv2
from PIL import Image

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchinfo import summary

from torchvision.models.efficientnet import efficientnet_b0, EfficientNet_B0_Weights

import albumentations as album
from albumentations.pytorch import ToTensorV2

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

warnings.filterwarnings("ignore")

# Set device
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {DEVICE}")

from sklearn.metrics import classification_report

from sklearn.metrics import (
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    roc_auc_score,
)

import time
import matplotlib.pyplot as plt
import torch.nn.functional as F
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, roc_curve
)
import numpy as np

import torch.nn.functional as F

In [ ]:
!rm -rf /kaggle/working/*

In [ ]:
source_base = "/kaggle/input/combined-dataset-pt1/C3_ShenzhenMontgomery_Classification_TB"
EPOCHS = 100
pat = 15

In [ ]:
import os
import shutil
from PIL import Image
from pathlib import Path

# Define source and destination base paths
dest_base = "/kaggle/working/data-resized"

# Define subdirectories
subdirs = [
    "test/Normal",
    "test/Tuberculosis",
    "train/Normal",
    "train/Tuberculosis"
]

# Desired image size (e.g., 256x256)
target_size = (256, 256)

# Create the directory structure
for subdir in subdirs:
    os.makedirs(os.path.join(dest_base, subdir), exist_ok=True)

# Function to resize and save image
def resize_and_save_image(src_path, dst_path, size):
    with Image.open(src_path) as img:
        img = img.resize(size, Image.LANCZOS)
        img.save(dst_path)

# Process each folder
for subdir in subdirs:
    src_folder = os.path.join(source_base, subdir)
    dst_folder = os.path.join(dest_base, subdir)

    for file_name in os.listdir(src_folder):
        src_file = os.path.join(src_folder, file_name)
        dst_file = os.path.join(dst_folder, file_name)

        if file_name.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp')):
            resize_and_save_image(src_file, dst_file, target_size)


In [ ]:
def is_image_file(filename):
    return filename.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.gif'))

def explore_directory(base_path):
    print(f"📂 Exploring: {base_path}\n")

    for root, dirs, files in os.walk(base_path):
        level = root.replace(base_path, '').count(os.sep)
        indent = '  ' * level
        sub_indent = '  ' * (level + 1)
        
        print(f"{indent}📁 {os.path.basename(root)}/")
        
        image_files = [f for f in files if is_image_file(f)]
        other_files = [f for f in files if not is_image_file(f)]
        
        if image_files:
            print(f"{sub_indent}🖼️  Image files: {len(image_files)}")
        if other_files:
            print(f"{sub_indent}📄 Other files: {len(other_files)}")
        if not image_files and not other_files:
            print(f"{sub_indent}(Empty folder)")

# Run the function
explore_directory('/kaggle/working/data-resized')

# Model Define

In [ ]:
# SeparableConv2d remains unchanged
class SeparableConv2d(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size=3, stride=1, padding=1, dilation=1, bias=False):
        super().__init__()
        self.depthwise = nn.Conv2d(in_channels, in_channels, kernel_size, stride, padding, dilation, groups=in_channels, bias=bias)
        self.pointwise = nn.Conv2d(in_channels, out_channels, kernel_size=1, bias=bias)

    def forward(self, x):
        return self.pointwise(self.depthwise(x))

# ASPP remains unchanged
class ASPP(nn.Module):
    def __init__(self, in_channels, out_channels, atrous_rates):
        super().__init__()
        modules = [
            nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=1, bias=False),
                nn.BatchNorm2d(out_channels),
                nn.ReLU(inplace=True)
            )
        ]
        for rate in atrous_rates:
            modules.append(nn.Sequential(
                SeparableConv2d(in_channels, out_channels, 3, padding=rate, dilation=rate, bias=False),
                nn.BatchNorm2d(out_channels),
                nn.ReLU(inplace=True)
            ))
        modules.append(nn.Sequential(
            nn.AdaptiveAvgPool2d(1),
            nn.Conv2d(in_channels, out_channels, 1, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True)
        ))
        self.convs = nn.ModuleList(modules)
        self.project = nn.Sequential(
            nn.Conv2d((len(atrous_rates) + 2) * out_channels, out_channels, 1, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
            nn.Dropout2d(0.5)
        )

    def forward(self, x):
        size = x.shape[2:]
        res = [F.interpolate(conv(x), size=size, mode='bilinear', align_corners=True) if i == len(self.convs)-1 else conv(x) for i, conv in enumerate(self.convs)]
        return self.project(torch.cat(res, dim=1))

# MFF Block
class MFFBlock(nn.Module):
    def __init__(self, in_channels_low, in_channels_high, out_channels):
        super().__init__()
        self.low_proj = nn.Conv2d(in_channels_low, out_channels, 1, bias=False)
        self.high_proj = nn.Conv2d(in_channels_high, out_channels, 1, bias=False)
        self.fusion = nn.Sequential(
            nn.Conv2d(out_channels, out_channels, 3, padding=1, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True)
        )
        self.se = nn.Sequential(
            nn.AdaptiveAvgPool2d(1),
            nn.Conv2d(out_channels, out_channels // 8, 1),
            nn.ReLU(),
            nn.Conv2d(out_channels // 8, out_channels, 1),
            nn.Sigmoid()
        )

    def forward(self, low_feat, high_feat):
        high_feat = F.interpolate(high_feat, size=low_feat.shape[2:], mode='bilinear', align_corners=True)
        low_feat = self.low_proj(low_feat)
        high_feat = self.high_proj(high_feat)
        x = low_feat + high_feat
        x = self.fusion(x)
        return x * self.se(x)

# CAFSE Block
class CAFSEBlock(nn.Module):
    def __init__(self, channels):
        super().__init__()
        self.coarse = nn.Sequential(
            nn.Conv2d(channels, channels, 3, padding=1),
            nn.BatchNorm2d(channels),
            nn.ReLU(inplace=True)
        )
        self.fine = nn.Sequential(
            nn.Conv2d(channels, channels, 1),
            nn.BatchNorm2d(channels),
            nn.Sigmoid()
        )

    def forward(self, decoder_feat, aspp_feat):
        aspp_feat = F.interpolate(aspp_feat, size=decoder_feat.shape[2:], mode='bilinear', align_corners=True)
        coarse = self.coarse(aspp_feat)
        fine = self.fine(decoder_feat)
        return decoder_feat + coarse * fine

# Decoder remains unchanged
class Decoder(nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.conv1 = nn.Sequential(
            nn.Conv2d(in_channels, 48, 1, bias=False),
            nn.BatchNorm2d(48),
            nn.ReLU(inplace=True)
        )
        self.fuse = nn.Sequential(
            SeparableConv2d(96, out_channels, 3, padding=1, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
            SeparableConv2d(out_channels, out_channels, 3, padding=1, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
            nn.Dropout2d(0.3)
        )

    def forward(self, x, low_level_feat):
        x = F.interpolate(x, size=low_level_feat.shape[2:], mode='bilinear', align_corners=True)
        x = self.conv1(x)
        x = torch.cat([x, low_level_feat], dim=1)
        return self.fuse(x)

# Main model
class DeepFusionLab(nn.Module):
    def __init__(self, num_classes_seg=1, num_classes_cls=2, mode=1, output_stride=16, activation='sigmoid'):
        super().__init__()
        self.mode = mode
        self.output_stride = output_stride

        backbone = efficientnet_b0(weights=EfficientNet_B0_Weights.IMAGENET1K_V1)
        features = list(backbone.features.children())
        if output_stride == 16:
            self.low_level = nn.Sequential(*features[:3])
            self.high_level = nn.Sequential(*features[3:])
        else:
            self.low_level = nn.Sequential(*features[:2])
            self.high_level = nn.Sequential(*features[2:])

        low_level_channels = 24 if output_stride == 16 else 16
        self.low_proj = nn.Sequential(
            nn.Conv2d(low_level_channels, 48, 1, bias=False),
            nn.BatchNorm2d(48),
            nn.ReLU(inplace=True)
        )

        atrous_rates = [6, 12, 18] if output_stride == 16 else [12, 24, 36]
        self.aspp = ASPP(1280, 256, atrous_rates)
        self.mff = MFFBlock(48, 256, 256)
        self.decoder = Decoder(256, 256)
        self.cafse = CAFSEBlock(256)
        self.final_conv = nn.Conv2d(256, num_classes_seg, 1)

        self.classifier = nn.Sequential(
            nn.AdaptiveAvgPool2d(1),
            nn.Flatten(),
            nn.Linear(1280, 512),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(512, num_classes_cls)
        )

        if activation == 'sigmoid':
            self.activation = nn.Sigmoid()
        elif activation == 'softmax2d':
            self.activation = nn.Softmax2d()
        else:
            self.activation = None

    def forward(self, x):
        input_size = x.size()[2:]
        low_feat = self.low_level(x)
        high_feat = self.high_level(low_feat)
        low_proj = self.low_proj(low_feat)

        if self.mode == 0:
            out = self.classifier(high_feat)
            return out
        elif self.mode == 1:
            aspp_out = self.aspp(high_feat)
            mff_out = self.mff(low_proj, aspp_out)
            decoder_out = self.decoder(mff_out, low_proj)
            cafse_out = self.cafse(decoder_out, aspp_out)
            out = self.final_conv(cafse_out)
            out = F.interpolate(out, size=input_size, mode='bilinear', align_corners=True)
            if self.activation is not None:
                out = self.activation(out)
            return out
        else:
            raise ValueError("Mode must be 0 (classification) or 1 (segmentation)")


In [ ]:
model = DeepFusionLab(num_classes_seg=2, num_classes_cls=2, mode=0)  # mode=1 for segmentation
model = model.to(DEVICE)

# Input image
input_tensor = torch.randn(2, 3, 256, 256).to(DEVICE)

# Forward pass
output = model(input_tensor)
print(output.shape)

def print_model_parameters(model):
    total_params = sum(p.numel() for p in model.parameters())
    trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    non_trainable_params = total_params - trainable_params

    print(f"Total Parameters: {total_params:,}")
    print(f"Trainable Parameters: {trainable_params:,}")
    print(f"Non-Trainable Parameters: {non_trainable_params:,}")

# Example usage
print_model_parameters(model)

summary(model, input_size=(2, 3, 256, 256))

# Dataframe

In [ ]:
# Define paths
data_dir_train = "/kaggle/working/data-resized/train"
data_dir_test = "/kaggle/working/data-resized/test"

# Load filepaths and labels
filepaths, labels = [], []

for fold in os.listdir(data_dir_train):
    foldpath = os.path.join(data_dir_train, fold)
    for file in os.listdir(foldpath):
        filepaths.append(os.path.join(foldpath, file))
        labels.append(fold)

# Create full training dataframe
train_df_full = pd.DataFrame({"image_path": filepaths, "label": labels})

# Split into train and validation sets
train_df, valid_df = train_test_split(train_df_full, test_size=0.2, random_state=42, stratify=train_df_full['label'])

# Load test data
test_filepaths, test_labels = [], []
for fold in os.listdir(data_dir_test):
    foldpath = os.path.join(data_dir_test, fold)
    for file in os.listdir(foldpath):
        test_filepaths.append(os.path.join(foldpath, file))
        test_labels.append(fold)

test_df = pd.DataFrame({"image_path": test_filepaths, "label": test_labels})


# Map class names to indices
class_to_idx = {cls: idx for idx, cls in enumerate(sorted(train_df['label'].unique()))}
print("Class to index mapping:", class_to_idx)

In [ ]:
# Custom Dataset Class
class MyDataGenerator(Dataset):
    def __init__(self, df, class_to_idx, augmentation=None, preprocessing=None):
        self.image_paths = df['image_path'].tolist()
        self.labels = df['label'].map(class_to_idx).tolist()
        self.augmentation = augmentation
        self.preprocessing = preprocessing

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, i):
        image = cv2.imread(self.image_paths[i])
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        label = self.labels[i]

        if self.augmentation:
            image = self.augmentation(image=image)['image']

        if self.preprocessing:
            image = self.preprocessing(image=image)['image']

        return image, label


# Augmentations
def get_training_augmentation():
    return album.Compose([
        album.HorizontalFlip(p=0.5),
        album.VerticalFlip(p=0.5),
        album.RandomRotate90(p=0.5),
        album.ShiftScaleRotate(shift_limit=0.0625, scale_limit=0.1, rotate_limit=15, p=0.5),
        album.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1, p=0.5),
    ])

def get_validation_augmentation():
    return album.Compose([
        album.PadIfNeeded(min_height=256, min_width=256, always_apply=True,
                           border_mode=cv2.BORDER_CONSTANT, value=0)
    ])

def to_tensor(x, **kwargs):
    return x.transpose(2, 0, 1).astype('float32')

def get_preprocessing():
    return album.Compose([
        album.Resize(height=256, width=256, always_apply=True),
        album.Lambda(image=to_tensor)
    ])

In [ ]:
# --- Compute Metrics Function ---
def compute_metrics(outputs, labels, calc_auc=False):
    _, preds = torch.max(outputs, 1)
    labels_cpu = labels.cpu()
    preds_cpu = preds.cpu()

    acc = accuracy_score(labels_cpu, preds_cpu)
    prec = precision_score(labels_cpu, preds_cpu, average='macro')
    rec = recall_score(labels_cpu, preds_cpu, average='macro')
    f1 = f1_score(labels_cpu, preds_cpu, average='macro')

    auc = float('nan')
    if calc_auc:
        probs = F.softmax(outputs, dim=1)[:, 1].detach().cpu().numpy()
        labels_np = labels_cpu.numpy()
        try:
            auc = roc_auc_score(labels_np, probs)
        except ValueError:
            pass

    return acc, prec, rec, f1, auc


class EarlyStopping:
    def __init__(self, patience=5, min_delta=0.001, metric='val_loss'):
        self.patience = patience
        self.min_delta = min_delta
        self.metric = metric
        self.best_score = None
        self.wait = 0
        self.stop_training = False

    def __call__(self, metrics_dict):
        raw_score = metrics_dict[self.metric]
        
        # Determine if this metric should be minimized (e.g., val_loss) or maximized (e.g., accuracy)
        minimize = self.metric == 'val_loss'
        current_score = -raw_score if minimize else raw_score

        # Initialize best score if first epoch
        if self.best_score is None:
            self.best_score = current_score
            self.wait = 0
        elif current_score < self.best_score + self.min_delta:
            self.wait += 1
            print(f"No improvement in {self.metric}. Wait: {self.wait}/{self.patience}")
            if self.wait >= self.patience:
                self.stop_training = True
                print("Early stopping triggered!")
        else:
            self.best_score = current_score
            self.wait = 0

        # Debug print in original metric scale
        best_raw = -self.best_score if minimize else self.best_score
        print(f"[DEBUG] {self.metric}: Current={raw_score:.6f}, Best={best_raw:.6f}")

# Dataloaders
train_dataset = MyDataGenerator(train_df, class_to_idx, get_training_augmentation(), get_preprocessing())
valid_dataset = MyDataGenerator(valid_df, class_to_idx, get_validation_augmentation(), get_preprocessing())
test_dataset = MyDataGenerator(test_df, class_to_idx, get_validation_augmentation(), get_preprocessing())

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True, num_workers=4)
valid_loader = DataLoader(valid_dataset, batch_size=8, shuffle=False, num_workers=4)
test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False, num_workers=4)


# Loss, Optimizer, Scheduler
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=1, T_mult=2, eta_min=1e-9)


# Train

In [ ]:
# --- Initialize Metric Trackers ---
train_losses, valid_losses = [], []
train_accs, valid_accs = [], []
train_precs, valid_precs = [], []
train_recs, valid_recs = [], []
train_f1s, valid_f1s = [], []

best_acc = 0.0
early_stopping = EarlyStopping(patience=pat, min_delta=1e-20, metric='val_loss')

start_time = time.time()

for epoch in range(EPOCHS):
    model.train()
    train_loss = 0.0
    train_acc = train_prec = train_rec = train_f1 = 0.0

    for images, labels in train_loader:
        images, labels = images.to(DEVICE), labels.to(DEVICE).long()
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        acc, prec, rec, f1, _ = compute_metrics(outputs, labels, calc_auc=False)
        train_loss += loss.item() * images.size(0)
        train_acc += acc * images.size(0)
        train_prec += prec * images.size(0)
        train_rec += rec * images.size(0)
        train_f1 += f1 * images.size(0)

    train_loss /= len(train_loader.dataset)
    train_acc /= len(train_loader.dataset)
    train_prec /= len(train_loader.dataset)
    train_rec /= len(train_loader.dataset)
    train_f1 /= len(train_loader.dataset)

    train_losses.append(train_loss)
    train_accs.append(train_acc)
    train_precs.append(train_prec)
    train_recs.append(train_rec)
    train_f1s.append(train_f1)

    model.eval()
    valid_loss = 0.0
    valid_acc = valid_prec = valid_rec = valid_f1 = 0.0

    with torch.no_grad():
        for images, labels in valid_loader:
            images, labels = images.to(DEVICE), labels.to(DEVICE).long()
            outputs = model(images)
            loss = criterion(outputs, labels)

            acc, prec, rec, f1, _ = compute_metrics(outputs, labels, calc_auc=False)
            valid_loss += loss.item() * images.size(0)
            valid_acc += acc * images.size(0)
            valid_prec += prec * images.size(0)
            valid_rec += rec * images.size(0)
            valid_f1 += f1 * images.size(0)

    valid_loss /= len(valid_loader.dataset)
    valid_acc /= len(valid_loader.dataset)
    valid_prec /= len(valid_loader.dataset)
    valid_rec /= len(valid_loader.dataset)
    valid_f1 /= len(valid_loader.dataset)

    valid_losses.append(valid_loss)
    valid_accs.append(valid_acc)
    valid_precs.append(valid_prec)
    valid_recs.append(valid_rec)
    valid_f1s.append(valid_f1)

    print(f"Epoch {epoch}: "
          f"Train Loss={train_loss:.4f}, Acc={train_acc:.4f}, Prec={train_prec:.4f}, Rec={train_rec:.4f}, F1={train_f1:.4f} | "
          f"Valid Loss={valid_loss:.4f}, Acc={valid_acc:.4f}, Prec={valid_prec:.4f}, Rec={valid_rec:.4f}, F1={valid_f1:.4f}")

    if valid_acc > best_acc:
        best_acc = valid_acc
        torch.save(model.state_dict(), 'Best_Weight.pth')
        print("Model saved!")

    metrics_dict = {'val_loss': valid_loss}
    early_stopping(metrics_dict)
    if early_stopping.stop_training:
        print(f"Training stopped at epoch {epoch}")
        break

    scheduler.step()

# --- Print Total Time ---
end_time = time.time()
elapsed = end_time - start_time
print(f"\nTotal training time: {elapsed/60:.2f} minutes")


In [ ]:
import matplotlib.pyplot as plt

# --- Subplots for Loss and Accuracy ---
fig, axes = plt.subplots(1, 2, figsize=(10, 3))  # 2 rows, 1 column

# Loss
axes[0].plot(train_losses, label='Train')
axes[0].plot(valid_losses, label='Validation')
axes[0].set_xlabel('Epoch')
axes[0].set_ylabel('Loss')
axes[0].set_title('Loss per Epoch')
axes[0].legend()
axes[0].grid(True)
axes[0].set_ylim(0, 1)  # optional, keep y-axis consistent

# Accuracy
axes[1].plot(train_accs, label='Train')
axes[1].plot(valid_accs, label='Validation')
axes[1].set_xlabel('Epoch')
axes[1].set_ylabel('Accuracy')
axes[1].set_title('Accuracy per Epoch')
axes[1].legend()
axes[1].grid(True)
axes[1].set_ylim(0, 1)

plt.tight_layout()
plt.show()


# Evaluation

In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import classification_report, confusion_matrix

# --------------------------
# Load model
# --------------------------
model.load_state_dict(torch.load('Best_Weight.pth'))
model.eval()

# --------------------------
# Compute predictions
# --------------------------
all_preds = []
all_labels = []

with torch.no_grad():
    for images, labels in test_loader:
        images = images.to(DEVICE)
        outputs = model(images)
        preds = torch.argmax(outputs, dim=1).cpu().numpy()
        all_preds.extend(preds)
        all_labels.extend(labels.numpy())

all_preds = np.array(all_preds)
all_labels = np.array(all_labels)

# --------------------------
# Classification report
# --------------------------
report = classification_report(
    all_labels, all_preds,
    target_names=list(class_to_idx.keys()),
    output_dict=True
)

# --- Print header ---
print(f"{'Class':15} {'Precision':>10} {'Recall':>10} {'F1-score':>10} {'Support':>10}")

# --- Print class rows ---
for cls in list(class_to_idx.keys()):
    metrics = report[cls]
    print(f"{cls:15} "
          f"{metrics['precision']:10.4f} "
          f"{metrics['recall']:10.4f} "
          f"{metrics['f1-score']:10.4f} "
          f"{metrics['support']:10.0f}")

# --- Print summary rows ---
print("\nOverall Metrics:")
for key in ["accuracy", "macro avg", "weighted avg"]:
    if key == "accuracy":
        print(f"{'Accuracy':15} {report[key]:10.4f}")
    else:
        metrics = report[key]
        print(f"{key:15} "
              f"{metrics['precision']:10.4f} "
              f"{metrics['recall']:10.4f} "
              f"{metrics['f1-score']:10.4f} "
              f"{metrics['support']:10.0f}")

# --------------------------
# Confusion matrix
# --------------------------
cm = confusion_matrix(all_labels, all_preds)
cm_normalized = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

class_names = list(class_to_idx.keys())

# Create annotations with "count (percent%)"
annot = np.empty_like(cm).astype(str)
for i in range(cm.shape[0]):
    for j in range(cm.shape[1]):
        annot[i, j] = f"{cm[i, j]} ({cm_normalized[i, j]*100:.2f}%)"

# Plot combined confusion matrix
plt.figure(figsize=(6, 6))
sns.heatmap(cm_normalized, annot=annot, fmt='', cmap='Blues',
            xticklabels=class_names, yticklabels=class_names)
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix (Count and Percentage)')
plt.show()

# --------------------------
# Misclassified indices
# --------------------------
misclassified_indices = np.where(all_preds != all_labels)[0]
print("Misclassified indices:", misclassified_indices)
